Базовый уровень: Генерация текста с использованием LSTM
Сбор данных: используйте готовый или соберите свой корпус в формате plain text для генерации текстов

Генерация текста на основе небольшого датасета

Предварительный анализ: чистка текста

Обучение модели. Используйте образец из туториала по RNNи
Генерация текста. Используйте образец из туториала по RNN
Сгенерируйте несколько текстов с помощью созданной модели

Я выбрала странный но интересный датасет на Кеггле - с русскими анекдотами (в превью вроде все прилично) https://www.kaggle.com/datasets/kdduha/russian-jokes

0. Подготовка: загружаем датасет, смотрим что там, вооружаемся библиотеками и тд.

In [2]:
# вооружаемся
import tensorflow as tf
import numpy as np
import pandas as pd
import sonnet as snt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [3]:
# Установка Kaggle API
!pip install -q kaggle

# Загрузка файла kaggle.json (предварительно загрузите его из своего аккаунта Kaggle)
from google.colab import files
uploaded = files.upload()

# Перемещение файла в нужную директорию
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Загрузка датасета
!kaggle datasets download -d kdduha/russian-jokes

# Распаковка архива
!unzip russian-jokes.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/kdduha/russian-jokes
License(s): MIT
russian-jokes.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  russian-jokes.zip
replace russian_jokes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: russian_jokes.csv       


In [4]:
df = pd.read_csv('russian_jokes.csv')  # посмотрим на загруженный через апи с кеггля датасет русскиз анекдотов (надеюсь, они приличные)
print(df.head(15))

                                                 text  comments_count  \
0   Директор отчитывает главбуха-еврея: \n- Яков С...               0   
1   — Изя, как вы думаете, объявления в газетах да...               0   
2   Метро. На единственное свободное место садится...               2   
3   Тюлька и килька вышли замуж за евреев. Теперь ...               2   
4   Судился еврей с армянином. В итоге судье дали ...               2   
5   — Вчера на меня напали грабители, отобрали кош...               0   
6   Встречаются два однокурсника:\n— Фима, не дава...               4   
7   Набережная. \nТурист потерявший кошелек громко...               0   
8   На одной из одесских улиц, а именно, на Дериба...               0   
9   Говорят, кошки ложатся на больное место. Сегод...               5   
10  — Яша, что ты целый час уже суетишься?\n— Не м...               5   
11  В Одесском зоопарке мать\nсыну:\n- Зямочка, не...               1   
12  Еврей вытаскивает из моря золотую рыбку. Она н.

In [5]:
# еще смотрим структурность дс, кол-во строк и т.д.
print("Информация о датасете:")
df.info()

Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63927 entries, 0 to 63926
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            61685 non-null  object
 1   comments_count  63927 non-null  int64 
 2   likes_count     63927 non-null  int64 
 3   reposts_count   63927 non-null  int64 
 4   views_count     63927 non-null  int64 
 5   date            63927 non-null  object
 6   unique_key      63927 non-null  object
 7   group_source    63927 non-null  object
dtypes: int64(4), object(4)
memory usage: 3.9+ MB


In [6]:
# нас интересуют конечно же сами анекдоты - выбираем их в данные, называем отдельно в дф
jokes = df['text'].tolist()
jokes[:15]

['Директор отчитывает главбуха-еврея: \n- Яков Самуилович, что за каменный век? У вас самый современный компьютер, а вы тарахтите счетами! Ну правда, перед клиентами стыдно! \n- Роман Аркадьевич, таки шо я вам имею сказать за бухгалтерию. Я уже закрыл не одну финансовую проверку, когда вы ходили пешком под стол и уверяю вас, счеты куда надежнее этого компьютера. Представьте - врывается налоговая в офис: "Всем к стене, руки за голову!". И когда вы будете удалять документы из компьютера? А я таки поднимаю руки со счетами, все косточки съезжают на одну сторону и все! Никто ничего не докажет!',
 '— Изя, как вы думаете, объявления в газетах дают результаты?\n— Конечно! В понедельник вышло объявление, что мы ищем сторожа, а уже во вторник нас ограбили!',
 'Метро. На единственное свободное место садится классический, как их описывают в анекдоте, еврей. Нос. Очки. Взгляд печального спаниеля. Обращается к соседу:\n— Вы еврей?\n— Почему?\n— Ваш ответ меня полностью удовлетворил.',
 'Тюлька и кил

1. Чистим дату

In [7]:

import re


# зачистка
def clean_joke(joke):
    if not isinstance(joke, str):  # проверка на нестроковые и NA
        return ""
    joke = ' '.join(joke.split()) # убиваем пробелы и переносы строк
    joke = re.sub(r'[^\w\s.,!?-]', '', joke) #убираем всякие символы
    return joke


cleaned_jokes = [clean_joke(joke) for joke in jokes]

# создаем новый дф для чистого материала
cleaned_df = pd.DataFrame({'original_text': jokes, 'cleaned_text': cleaned_jokes})

# проверка
for i, joke in enumerate(cleaned_jokes[:5], 1):
    print(f"{i}. {joke}\n")

1. Директор отчитывает главбуха-еврея - Яков Самуилович, что за каменный век? У вас самый современный компьютер, а вы тарахтите счетами! Ну правда, перед клиентами стыдно! - Роман Аркадьевич, таки шо я вам имею сказать за бухгалтерию. Я уже закрыл не одну финансовую проверку, когда вы ходили пешком под стол и уверяю вас, счеты куда надежнее этого компьютера. Представьте - врывается налоговая в офис Всем к стене, руки за голову!. И когда вы будете удалять документы из компьютера? А я таки поднимаю руки со счетами, все косточки съезжают на одну сторону и все! Никто ничего не докажет!

2.  Изя, как вы думаете, объявления в газетах дают результаты?  Конечно! В понедельник вышло объявление, что мы ищем сторожа, а уже во вторник нас ограбили!

3. Метро. На единственное свободное место садится классический, как их описывают в анекдоте, еврей. Нос. Очки. Взгляд печального спаниеля. Обращается к соседу  Вы еврей?  Почему?  Ваш ответ меня полностью удовлетворил.

4. Тюлька и килька вышли замуж з

In [8]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63927 entries, 0 to 63926
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   original_text  61685 non-null  object
 1   cleaned_text   63927 non-null  object
dtypes: object(2)
memory usage: 999.0+ KB


In [10]:
corp = cleaned_jokes[:len(cleaned_jokes)//15] # сократим чтобы не нервировать Cpu в гугл колабе и не ждать 100500 часов

In [11]:
lower_corp = [text.lower() for text in corp]
print(lower_corp[20])

крупный фабрикант приходит к раввину  ребе, у меня проблемы. фабрика приносит одни убытки, дисциплины никакой, производительность на нуле, долги растут, налоги заели. что делать?  возьми талмуд, положи его подмышку и обходи всю фабрику два раза в день. через месяц приходит радостный фабрикант к раввину и говорит  замечательно, воровство на работе прекратилось, бездельники уволены, производительность выросла, с долгами покончено! в чем секрет?  руководитель должен постоянно находиться у себя на производстве и вникать во все, что происходит.  это я понял. а талмуд зачем?  для солидности...


2. Обучаем

In [12]:
# инициация токенайзера
tokenizer = Tokenizer(num_words=10000, oov_token="")
tokenizer.fit_on_texts(lower_corp)
sequences = tokenizer.texts_to_sequences(lower_corp)

In [13]:
tokenizer.texts_to_sequences([lower_corp[0]])[0]

[1339,
 8232,
 8233,
 60,
 363,
 8234,
 8,
 15,
 8235,
 8236,
 10,
 37,
 274,
 8237,
 1449,
 6,
 9,
 8238,
 4950,
 25,
 224,
 156,
 6422,
 1107,
 3176,
 8239,
 17,
 21,
 7,
 49,
 1340,
 210,
 15,
 8240,
 7,
 43,
 688,
 4,
 357,
 6423,
 8241,
 50,
 9,
 2377,
 3869,
 151,
 739,
 2,
 2126,
 37,
 8242,
 358,
 8243,
 190,
 4951,
 4952,
 6424,
 3177,
 3,
 1450,
 325,
 19,
 825,
 464,
 15,
 530,
 2,
 50,
 9,
 229,
 3178,
 1252,
 28,
 4951,
 6,
 7,
 17,
 4953,
 464,
 139,
 4950,
 33,
 8244,
 8245,
 5,
 357,
 875,
 2,
 33,
 551,
 100,
 4,
 8246]

In [14]:
X = []
y = []
MAX_SAMPLES = 30000

for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])
        if len(X) >= MAX_SAMPLES:
            break
    if len(X) >= MAX_SAMPLES:
        break

In [15]:
class RNN(snt.RNNCore):

  # инициация параметров RNN
  def __init__(self, hidden_size, activation=tf.tanh, name="vanilla_rnn"):
    """
    hidden_size: размер скрытого слоя
    activation: тип функции активации
    name: название модели
    """
    # вызов конструктора родительского класса (snt.RNNCore)
    super(RNN, self).__init__(name=name)

    # сохранение размера скрытого состояния
    self._hidden_size = hidden_size

    # сохранение активации
    self._activation = activation

In [16]:
def _build(self, input_, prev_state):
    """
    input_: тензор с текущим x_t
    prev_tate: тезор с h_{t-1}
    """
    # Создаем линейный слой для преобразования входных данных в скрытое состояние
    self._in_to_hidden_linear = snt.Linear(
        self._hidden_size, name="in_to_hidden")

    # Создаем линейный слой для преобразования предыдущего скрытого состояния в новое
    self._hidden_to_hidden_linear = snt.Linear(
        self._hidden_size, name="hidden_to_hidden")

    # Применяем линейное преобразование к входным данным
    in_to_hidden = self._in_to_hidden_linear(input_)

    # Применяем линейное преобразование к предыдущему скрытому состоянию
    hidden_to_hidden = self._hidden_to_hidden_linear(prev_state)

    # Складываем результаты и применяем функцию активации
    output = self._activation(in_to_hidden + hidden_to_hidden)

    # Возвращаем выходное значение и новое скрытое состояние
    return output, output

# Свойство state_size возвращает размерность скрытого состояния
@property
def state_size(self):
    return tf.TensorShape([self._hidden_size])

# Свойство output_size возвращает размерность выходного значения
@property
def output_size(self):
    return tf.TensorShape([self._hidden_size])

In [17]:
from keras.utils import to_categorical
import numpy as np

max_sequence_len = max([len(x) for x in X])
X = pad_sequences(X, maxlen=max_sequence_len, padding='pre')
y = np.array(y)
y = to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [18]:

# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_len))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Теперь можно обучать модель
# экстра коммент - сначала я уменьшила размер на 3, потом на4, потом на 5 и так до 15 - только так норм заработало(хотя все равно долговато)... но может оно и к лучшему :) вышло 3~ часа
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 231s 609ms/step - accuracy: 0.0274 - loss: 8.3403 - val_accuracy: 0.0298 - val_loss: 7.7864
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 264s 616ms/step - accuracy: 0.0310 - loss: 7.3010 - val_accuracy: 0.0448 - val_loss: 7.8659
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 221s 591ms/step - accuracy: 0.0476 - loss: 7.0032 - val_accuracy: 0.0518 - val_loss: 7.9471
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 253s 567ms/step - accuracy: 0.0586 - loss: 6.6968 - val_accuracy: 0.0502 - val_loss: 8.0284
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 218s 582ms/step - accuracy: 0.0601 - loss: 6.4086 - val_accuracy: 0.0502 - val_loss: 8.1290
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 259s 574ms/step - accuracy: 0.0687 - loss: 6.0838 - val_accuracy: 0.0547 - val_loss: 8.2271
Epoch 7/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 260s 569ms/step - accuracy: 0.0745 - loss: 5.7575 - val_accuracy: 0.0570 - val_loss: 8.3299
Epoch 8/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 262s 569ms/step - accuracy: 0.0912 -

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 222, 100)         │     1,491,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (64, 150)              │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 14916)            │     2,252,316 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,683,550 (44.57 MB)

 Trainable params: 3,894,516 (14.86 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,789,034 (29.71 MB)

In [22]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += ' ' + output_word
    return seed_text

generated = generate_text("товарищ", 25, max_sequence_len)
print(generated)

товарищ рабинович вы любите родину конечно люблю от всей души а вы готовы отдать за нее жизнь вы меня конечно извините но кто же тогда будет


In [25]:
# еще генерируем
def generate_multiple_texts(seed_texts, next_words, max_sequence_len):
    generated_texts = []
    for seed_text in seed_texts:
        generated_text = generate_text(seed_text, next_words, max_sequence_len)
        generated_texts.append(generated_text)
    return generated_texts

seed_texts = [
    "мой",
    "город",
    "я",
    "документ",
    "едет",
    "стоматолог",
    "здесь"
]

generated_texts = generate_multiple_texts(seed_texts, next_words=25, max_sequence_len=max_sequence_len)

for i, generated_text in enumerate(generated_texts):
    print(f"Generated text {i+1}:\n{generated_text}\n")


Generated text 1:
мой в израильском фейсбуке выйду замуж за еврея любой национальности сдохшего льва и скажите ну что честно они тебя не стоишь а шо вы то на

Generated text 2:
город на памятнике здесь покоится известный одесский стоматолог борис рафаилович кац а его сын моня принимает в его кабинете на прохоровской 21 что случилось люди на

Generated text 3:
я передумал и в израиль не поеду вы же уже документы получили тур оплатили не поеду я но почему там плюс 40 в тени ну а

Generated text 4:
документ в                        

Generated text 5:
едет на тот напали грабители на пороге смотрит и начинает показать язык есть больше я не обоев покупали когда делали свой ремонт он пошел ему деньги

Generated text 6:
стоматолог еврея мюнхен сильно пострадал от голубей в результате эпидемии у птиц случилась диарея и голубиный помет усеял крыши мэрия мюнхена ежедневно тратила большие суммы для

Generated text 7:
здесь е годы рабинович в                     



In [26]:

# Сохраняем модель
model.save('news_headline_generator.keras')

Выводы:
очень интересный и забавный эксперимент получился - пришлось подбирать и максимальные токены, и объем датасета. Наверное уже потом вне курса попробую что-то подобное, но в реалиях более длинного промежутка времени на обучения.
Видно, что точность недотягивает в моем итоговом варианте - но в большинстве случае с более трудоемкими настройками среды вылетала, так что довольствуюсь малым.
В целом - модель хорошо уловила стиль датасета - немного больше "еврейской" тематики чем я ожидала. Конечно, чем менее частотное слово, тем меньше контектса он ловит и генерирует большую околесицу. Любопытно, что в 6 примере дважды был мюнхен, и даже была последовательность вполне читаемая (хотя и не очень приличная)
По обучению - оноо получилось не очень эффективно, но была возможность оставить думать на 3,5 часа - после ухищрений с длиной датасета, эпохами, токенами. В целом, получилось забавно и стиль "русского анекдота" модель сохранила при генерации.